In [1]:
import pandas as pd

In [2]:
# this notebook merges data about us companies from multiple csvs, so we can calculate necessary ratios
income = pd.read_csv('./data/us-income-annual.csv', sep=';')
share_prices = pd.read_csv('./data/us-shareprices-daily.csv', sep=';')
cashflow = pd.read_csv('./data/us-cashflow-annual.csv', sep=';')
balance = pd.read_csv('./data/us-balance-annual.csv', sep=';')

In [3]:
# income should only contribute unique features, and features to merge on
unique_income_cols = income.columns.difference(cashflow.columns)
print('Income contributing:\n', unique_income_cols)
unique_income_cols = unique_income_cols.insert(0, 'Ticker')
unique_income_cols = unique_income_cols.insert(0, 'Fiscal Year')

Income contributing:
 Index(['Abnormal Gains (Losses)', 'Cost of Revenue', 'Gross Profit',
       'Income (Loss) from Continuing Operations',
       'Income Tax (Expense) Benefit, Net', 'Interest Expense, Net',
       'Net Extraordinary Gains (Losses)', 'Net Income', 'Net Income (Common)',
       'Non-Operating Income (Loss)', 'Operating Expenses',
       'Operating Income (Loss)', 'Pretax Income (Loss)',
       'Pretax Income (Loss), Adj.', 'Research & Development', 'Revenue',
       'Selling, General & Administrative'],
      dtype='object')


In [4]:
cashflow_income = pd.merge(cashflow, income[unique_income_cols], how='outer',
                           left_on=['Ticker', 'Fiscal Year'],
                           right_on=['Ticker', 'Fiscal Year'])

In [5]:
# balance should only contribute unique features, and features to merge on
balance_unique_columns = balance.columns.difference(cashflow_income.columns)
print('Balance contributing:\n', balance_unique_columns)
balance_unique_columns = balance_unique_columns.insert(0, 'Ticker')
balance_unique_columns = balance_unique_columns.insert(0, 'Fiscal Year')

Balance contributing:
 Index(['Accounts & Notes Receivable',
       'Cash, Cash Equivalents & Short Term Investments', 'Inventories',
       'Long Term Debt', 'Long Term Investments & Receivables',
       'Other Long Term Assets', 'Payables & Accruals',
       'Property, Plant & Equipment, Net', 'Retained Earnings',
       'Share Capital & Additional Paid-In Capital', 'Short Term Debt',
       'Total Assets', 'Total Current Assets', 'Total Current Liabilities',
       'Total Equity', 'Total Liabilities', 'Total Liabilities & Equity',
       'Total Noncurrent Assets', 'Total Noncurrent Liabilities',
       'Treasury Stock'],
      dtype='object')


In [6]:
cashflow_income_balance = pd.merge(cashflow_income, balance[balance_unique_columns], how='outer',
                                   left_on=['Ticker', 'Fiscal Year'],
                                   right_on=['Ticker', 'Fiscal Year'])

In [14]:
not_needed = ['Currency', 'Fiscal Period', 'Publish Date', 'Restated Date',
              'Depreciation & Amortization', 'Non-Cash Items', 'Change in Accounts Receivable',
              'Change in Inventories', 'Change in Accounts Payable', 'Change in Other',
              'Net Cash from Operating Activities', 'Change in Fixed Assets & Intangibles',
              'Net Cash from Acquisitions & Divestitures', 'Net Cash from Investing Activities', 'Dividends Paid',
              'Cash from (Repayment of) Debt', 'Cash from (Repurchase of) Equity', 'Net Cash from Financing Activities',
              'Abnormal Gains (Losses)', 'Income (Loss) from Continuing Operations',
              'Income Tax (Expense) Benefit, Net',
              'Interest Expense, Net', 'Net Extraordinary Gains (Losses)', 'Net Income (Common)',
              'Non-Operating Income (Loss)', 'Accounts & Notes Receivable', 'Inventories', 'Other Long Term Assets',
              'Payables & Accruals', 'Property, Plant & Equipment, Net']
needed_feats = cashflow_income_balance.drop(columns=not_needed)
print('Needed features to calculate ratios:\n:', needed_feats.columns.to_list())

Index(['Ticker', 'SimFinId', 'Fiscal Year', 'Report Date', 'Shares (Basic)',
       'Shares (Diluted)', 'Net Income/Starting Line',
       'Change in Working Capital', 'Net Change in Long Term Investment',
       'Net Change in Cash', 'Cost of Revenue', 'Gross Profit', 'Net Income',
       'Operating Expenses', 'Operating Income (Loss)', 'Pretax Income (Loss)',
       'Pretax Income (Loss), Adj.', 'Research & Development', 'Revenue',
       'Selling, General & Administrative',
       'Cash, Cash Equivalents & Short Term Investments', 'Long Term Debt',
       'Long Term Investments & Receivables', 'Retained Earnings',
       'Share Capital & Additional Paid-In Capital', 'Short Term Debt',
       'Total Assets', 'Total Current Assets', 'Total Current Liabilities',
       'Total Equity', 'Total Liabilities', 'Total Liabilities & Equity',
       'Total Noncurrent Assets', 'Total Noncurrent Liabilities',
       'Treasury Stock'],
      dtype='object')

In [8]:
cashflow_income_balance.to_csv('cashflow_income_balance_feats.csv')

In [17]:
# merge in adjusted closing prices on december 31
shareprices = pd.read_csv('./data/us-shareprices-daily.csv', sep=';')

In [24]:
adj_close = shareprices[['Ticker', 'SimFinId', 'Date', 'Adj. Close']]
shareprices_filtered = adj_close[adj_close['Date'].str.endswith('12-31')]
shareprices_filtered['Year'] = shareprices_filtered['Date'].str.slice(0, 4)
shareprices_filtered['Year'] = shareprices_filtered['Year'].astype(int)
shareprices_filtered.drop(columns=['Date', 'SimFinId'], inplace=True)

<ipython-input-24-de3da5f1a556>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shareprices_filtered['Year'] = shareprices_filtered['Date'].str.slice(0, 4)
<ipython-input-24-de3da5f1a556>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shareprices_filtered['Year'] = shareprices_filtered['Year'].astype(int)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

In [40]:
# no data from 2011, 2016, 2017

(shareprices_filtered['Year'] == 2017).sum()

0

In [49]:
cashflow_income_balance_without_missing = cashflow_income_balance[cashflow_income_balance['Fiscal Year'] != 2011]
cashflow_income_balance_without_missing = cashflow_income_balance_without_missing[
    cashflow_income_balance['Fiscal Year'] != 2016]
cashflow_income_balance_without_missing = cashflow_income_balance_without_missing[
    cashflow_income_balance['Fiscal Year'] != 2017]

<ipython-input-49-8589638311b3>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cashflow_income_balance_without_missing = cashflow_income_balance_without_missing[cashflow_income_balance['Fiscal Year'] != 2016]
<ipython-input-49-8589638311b3>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cashflow_income_balance_without_missing = cashflow_income_balance_without_missing[cashflow_income_balance['Fiscal Year'] != 2017]


In [50]:
cashflow_income_balance_shareprices = pd.merge(cashflow_income_balance_without_missing, shareprices_filtered,
                                               how='outer',
                                               left_on=['Ticker', 'Fiscal Year'],
                                               right_on=['Ticker', 'Year'])

In [51]:
# drop companies without closing prices
cashflow_income_balance_shareprices_x = cashflow_income_balance_shareprices[
    cashflow_income_balance_shareprices['Adj. Close'].notna()]

In [52]:
cashflow_income_balance_shareprices_x.to_csv('features_target_before_ratios.csv')